In [1]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup

In [2]:
#pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [18]:
# Ejecutando el código comentado sale una pestaña de más y un warning
#options = webdriver.ChromeOptions()
#options.add_argument('user-agent = Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36')
#driver = webdriver.Chrome(chrome_options=options)
#driver.get("https://genealogy.math.ndsu.nodak.edu/")

options_chrome = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = options_chrome)
driver.get("https://genealogy.math.ndsu.nodak.edu/")

In [19]:
# Clicar para la búsqueda avanzada en "Advanced Search"
advance_search = driver.find_element(By.XPATH, '//a[@href="search.php"]')
advance_search.click()

# Rellenar los campos necesarios y hacer enter
given_name_search = driver.find_element(By.ID, 'given_name')
given_name_search.send_keys("Ma")

subject_search = driver.find_element(By.XPATH, '//option[@value="62"]')
subject_search.click()

country_search = driver.find_element(By.ID, 'country')
country_search.send_keys("Spain" + Keys.ENTER)

In [20]:
#cargamos el contenido (ya se podría cerrar la pestaña)
soup = BeautifulSoup(driver.page_source)

In [21]:
main_content = soup.find(id='mainContent').table
authors_searched = main_content.find_all('tr')
print(authors_searched)

[<tr><td><a href="id.php?id=213317">Aguilera Morillo, María del Carmen</a></td>
<td>Universidad de Granada</td>
<td>2013</td></tr>, <tr><td><a href="id.php?id=271091">Almuzara, Martín</a></td>
<td>Universidad Internacional Menéndez Pelayo</td>
<td>2020</td></tr>, <tr><td><a href="id.php?id=265694">Cabero, Ismael</a></td>
<td>Universitat Jaume I de Castellón</td>
<td>2019</td></tr>, <tr><td><a href="id.php?id=210494">Casals, María Rosa</a></td>
<td>Universidad de Cantabria</td>
<td>1988</td></tr>, <tr><td><a href="id.php?id=268419">Ciccarelli, Matteo</a></td>
<td>Universitat Pompeu Fabra</td>
<td>2001</td></tr>, <tr><td><a href="id.php?id=213324">Del Moral Ávila, María José</a></td>
<td>Universidad de Granada</td>
<td>1993</td></tr>, <tr><td><a href="id.php?id=183227">Dominguez-Toribio, Manuel Angel</a></td>
<td>Universidad Carlos III de Madrid</td>
<td>1998</td></tr>, <tr><td><a href="id.php?id=213318">Escabias Machuca, Manuel</a></td>
<td>Universidad de Granada</td>
<td>2002</td></tr>

In [22]:
import re

# función que dada una lista de autores extraída de la búsqueda devuelve sus ids
# para luego hacer peticiones con cada id a https://genealogy.math.ndsu.nodak.edu/id.php?id=<id>
# y de ahí extraer la info
def authors_ids(authors_searched):
    ids = []
    for author in authors_searched:
        link = author.find('a').get('href')
        id = re.search('id=(.*)', link).group(1)
        ids.append(id)
    return ids

In [23]:
print(authors_ids(authors_searched))

['213317', '271091', '265694', '210494', '268419', '213324', '183227', '213318', '129437', '210498', '210638', '215454', '35146', '254763', '213349', '150826', '108099', '210661', '210493', '163488', '84571', '210655', '210533', '150829', '210660', '210492', '170359', '183292', '153482', '150828', '210656', '210641']


In [24]:
def authors_information(authors_id):
    """
    Realiza web scraping en las pestañas de los autores
    con id en 'authors_id'.
    
    Para cada id se scrapea la página de ese autor. Mediante las
    funciones principales de BeautifulSoup y las expresiones
    regulares, se van guardando, en este orden: nombre, universidad,
    año, pais, titulo, tutores y estudiantes.
    Finalmente, se crea una lista con los valores de estos campos y
    se añade a la lista 'information'.
    
    Parámetros:
        authors_id: lista de los ids de los autores obtenidos según
        la búsqueda

    Devuelve:
        information: lista anidada, en cada una la información de un autor/a.
    """
    #information = [] #Para CSV
    data = {}
    data['Mathematicians'] = []
    for identificador in authors_id:
        # LECTURA DE LA PÁGINA
        urls = driver.get("https://genealogy.math.ndsu.nodak.edu/id.php?id="+identificador)
        soup = BeautifulSoup(driver.page_source)

        # WEB SCRAPING
        main_content2 = soup.find(id='mainContent')
        
        # Nombre autor/autora
        nombre = main_content2.find('h2').get_text().strip() #strip() elimina los espacios en blanco al inicio y al final
        
        # Universidad
        universidad = main_content2.find_next('span').find_next('span').get_text()
        
        # Año de publicación de la tesis
        año = main_content2.find('span').get_text()
        año = ''.join(re.findall(r"\d{4}$", año))
        
        # País
        if main_content2.find('img'):
            pais = main_content2.find('img').get('title')
        else:
            pais = ''
            
        for div in range(0, len(main_content2.find_all('div'))):
            # Título de la tesis
            if div==3:
                # PRIMERA OPCIÓN
                #titulo = main_content2.find_all('div')[div].find_next('span').find_next('span').get_text().strip()
                # SEGUNDA OPCIÓN
                titulo = ''.join(main_content2.find_all('div')[div].contents[2].contents).strip()
            # Asignatura
            if div == 4:
                asignatura = main_content2.find_all('div')[div].get_text().split(r":", 1)[1].strip()
            else:
                asignatura=''

        # Tutor/Tutores
        advisor_text = re.search(r'Advisor.*',main_content2.get_text()).group(0)
        advisors_pre = re.split(r"No students known.", advisor_text)[0]
        advisors = re.split(r"Advisor: |Advisor \d: ",advisors_pre)[1:]

        # Estudiantes
        students = []
        if main_content2.table != None:
            students_searched = main_content2.table.find_all('a')
            for stud in students_searched:
                students.append(stud.get_text())

        # Para CSV
        #elemento = [nombre, universidad, año, pais, titulo, asignatura, advisors, students]
        #information.append(elemento)
        
        # Para JSON
        data['Mathematicians'].append({
            'Nombre': nombre,
            'Universidad': universidad,
            'Año': año,
            'País': pais,
            'Título': titulo,
            'Asignatura': asignatura,
            'Tutores': advisors,
            'Estudiantes': students
        })
    return data

In [25]:
info = authors_information(authors_ids(authors_searched))
info

{'Mathematicians': [{'Nombre': 'María del Carmen  Aguilera Morillo',
   'Universidad': 'Universidad de Granada',
   'Año': '2013',
   'País': 'Spain',
   'Título': 'Penalized estimation methods in functional data analysis',
   'Asignatura': '62—Statistics',
   'Tutores': ['Ana María Aguilera del Pino'],
   'Estudiantes': []},
  {'Nombre': 'Martín  Almuzara',
   'Universidad': 'Universidad Internacional Menéndez Pelayo',
   'Año': '2020',
   'País': 'Spain',
   'Título': 'Essays on Latent Variables in Time Series and Panel Data',
   'Asignatura': '62—Statistics',
   'Tutores': ['Manuel  Arellano', 'Enrique  Sentana'],
   'Estudiantes': []},
  {'Nombre': 'Ismael  Cabero',
   'Universidad': 'Universitat Jaume I de Castellón',
   'Año': '2019',
   'País': 'Spain',
   'Título': 'Some contributions to archetypal analysis with applications',
   'Asignatura': '62—Statistics',
   'Tutores': ['Irene  Epifanio'],
   'Estudiantes': []},
  {'Nombre': 'María Rosa  Casals',
   'Universidad': 'Univers

In [26]:
# GUARDAR LOS DATOS EN JSON
import json
with open('mathematicians_dataset.json', 'w') as file:
    json.dump(info, file, indent=4)

In [27]:
with open("mathematicians_dataset.json", "r") as j:
    mydata = json.load(j)
    print(mydata)

{'Mathematicians': [{'Nombre': 'María del Carmen  Aguilera Morillo', 'Universidad': 'Universidad de Granada', 'Año': '2013', 'País': 'Spain', 'Título': 'Penalized estimation methods in functional data analysis', 'Asignatura': '62—Statistics', 'Tutores': ['Ana María Aguilera del Pino'], 'Estudiantes': []}, {'Nombre': 'Martín  Almuzara', 'Universidad': 'Universidad Internacional Menéndez Pelayo', 'Año': '2020', 'País': 'Spain', 'Título': 'Essays on Latent Variables in Time Series and Panel Data', 'Asignatura': '62—Statistics', 'Tutores': ['Manuel  Arellano', 'Enrique  Sentana'], 'Estudiantes': []}, {'Nombre': 'Ismael  Cabero', 'Universidad': 'Universitat Jaume I de Castellón', 'Año': '2019', 'País': 'Spain', 'Título': 'Some contributions to archetypal analysis with applications', 'Asignatura': '62—Statistics', 'Tutores': ['Irene  Epifanio'], 'Estudiantes': []}, {'Nombre': 'María Rosa  Casals', 'Universidad': 'Universidad de Cantabria', 'Año': '1988', 'País': 'Spain', 'Título': 'Contraste

In [13]:
# GUARDAR LOS DATOS EN CSV
import csv
headers = ['Nombre', 'Universidad', 'Año', 'País', 'Título', 'Asignatura', 'Tutores', 'Estudiantes']

# OPCIÓN 1 CSV
with open('mathematicians_dataset.csv', 'w', encoding='UTF8', newline='') as csvfile:
    for column in range(len(headers)):
        csvfile.write((headers[column]) + ";")
    csvfile.write("\n");
    for i in range(len(info)):
        for j in range(len(info[i])):
            csvfile.write(str(info[i][j]) + ";");
        csvfile.write("\n");

In [14]:
# OPCIÓN 2 CSV (Las dos opciones salen iguales)
with open('mathematicians_dataset2.csv', 'w', encoding='UTF8', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(headers)
    writer.writerows(info)

In [15]:
data = pd.read_csv("mathematicians_dataset2.csv", sep=";")
print(data)

                                   Nombre  \
0  Apépé Jugeandène  Nguengue Louvouandou   
1                           James T. Pepe   
2                  Margaret Sullivan Pepe   
3                     Wilfrid Dennis Pepe   
4                           William  Pepe   
5                         Aljoša  Peperko   
6                     Jan A. Peperstraete   
7                 Jose (Pepe) M. Quintana   

                             Universidad     Año  \
0              Marien Ngouabi University  2019.0   
1  Massachusetts Institute of Technology  1972.0   
2               University of Washington  1986.0   
3                     Indiana University  1966.0   
4              University of Connecticut  2007.0   
5                University of Ljubljana  2008.0   
6                                    NaN     NaN   
7                  University of Warwick  1987.0   

                               País  \
0  Democratic Republic of the Congo   
1                      UnitedStates   
2       